In [91]:
import pandas as pd
import yfinance as yf
import numpy as np
from arch import arch_model
from scipy.stats import norm

# Function to calculate the Black-Scholes option price
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == 'put':
        price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return price, d1, d2

# Function to calculate Greeks
def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    price, d1, d2 = black_scholes_price(S, K, T, r, sigma, option_type)

    if option_type == 'call':
        delta = norm.cdf(d1)
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return delta, gamma

# Function to calculate annualized volatility using the last 60 days
def calculate_annualized_volatility(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['Log Returns'] = np.log(stock_data['Adj Close'] / stock_data['Adj Close'].shift(1))
    rolling_volatility = stock_data['Log Returns'].rolling(window=60).std()
    annualized_volatility = rolling_volatility * np.sqrt(252)    
    return annualized_volatility

# Function to calculate GARCH volatility using the last 60 days
def calculate_garch_volatility(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data['Log Returns'] = np.log(stock_data['Adj Close'] / stock_data['Adj Close'].shift(1))
    returns = stock_data['Log Returns'].dropna()
    garch_model = arch_model(returns, vol='Garch', p=1, q=1)
    garch_fit = garch_model.fit(disp='off')
    garch_volatility = garch_fit.conditional_volatility
    annualized_garch_volatility = garch_volatility * np.sqrt(252)
    return annualized_garch_volatility

# Read Excel file
file_path = "D:/trial&error/Black_Scholes.xlsx"
df = pd.read_excel(file_path)

# Concatenate ".NS" to each stock name
df['Stock'] = df['Stock'].astype(str) + ".NS"

# Create new columns for storing the calculated values
df['Annualized Volatility'] = np.nan
df['GARCH Volatility'] = np.nan
df['Trading Days to Expiry'] = np.nan
df['Current Stock Price'] = np.nan
df['Theoretical Option Price'] = np.nan
df['GARCH Theoretical Option Price'] = np.nan  # New column for GARCH option price
df['Delta'] = np.nan
df['Gamma'] = np.nan

# Processing the data
for index, row in df.iterrows():
    stock = row['Stock']
    date = pd.to_datetime(row['Date'])
    expiry_date = pd.to_datetime(row['Expiry Date'])
    
    # Use the previous 60 days from the current row's date to calculate volatility
    start_date = (date - pd.DateOffset(days=90)).strftime('%Y-%m-%d')  
    end_date = date.strftime('%Y-%m-%d')
    
    try:
        annualized_vol = calculate_annualized_volatility(stock, start_date, end_date)
        df.at[index, 'Annualized Volatility'] = annualized_vol[-1]  # Get the last volatility value
        garch_vol = calculate_garch_volatility(stock, start_date, end_date)
        df.at[index, 'GARCH Volatility'] = garch_vol[-1]  # Get the last GARCH volatility value
        trading_days = np.busday_count(date.date(), expiry_date.date())
        df.at[index, 'Trading Days to Expiry'] = trading_days
        current_price = yf.download(stock, start=date.strftime('%Y-%m-%d'), end=(date + pd.DateOffset(days=1)).strftime('%Y-%m-%d'))['Adj Close'].iloc[0]
        df.at[index, 'Current Stock Price'] = current_price
        K = row['Strike']  # Strike price
        T = trading_days / 252  # Convert trading days to years
        r = 0.1  # Risk-free rate (10%)
        sigma = annualized_vol[-1]  # Use historical volatility

        option_type = 'call' if row['CE/PE'] == 'CE' else 'put'
        
        option_price = black_scholes_price(current_price, K, T, r, sigma, option_type)
        df.at[index, 'Theoretical Option Price'] = option_price[0]
        garch_sigma = garch_vol[-1]  # Use GARCH volatility
        garch_option_price = black_scholes_price(current_price, K, T, r, garch_sigma, option_type)
        df.at[index, 'GARCH Theoretical Option Price'] = garch_option_price[0]

        # Calculate Greeks
        delta, gamma = calculate_greeks(current_price, K, T, r, sigma, option_type)
        df.at[index, 'Delta'] = delta
        df.at[index, 'Gamma'] = gamma

    except Exception as e:
        print(f"Could not calculate for {stock} on {date}: {e}")

# Save the updated DataFrame back to the Excel file
with pd.ExcelWriter("D:/trial&error/bsm.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, index=False)


[*********************100%***********************]  1 of 1 completed
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24312\2951366787.py:84: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.at[index, 'Annualized Volatility'] = annualized_vol[-1]  # Get the last volatility value
[*********************100%***********************]  1 of 1 completed
C:\Users\LENOVO\anaconda3\Lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001812. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\U

In [92]:
df

,Date,Stock,CE/PE,Strike,Expiry Date,Annualized Volatility,GARCH Volatility,Trading Days to Expiry,Current Stock Price,Theoretical Option Price,GARCH Theoretical Option Price,Delta,Gamma
0,2024-09-21,INFY.NS,CE,1900,2024-09-26,0.218492,0.220185,3.0,1910.000000,24.893328,25.028619,0.611049,0.008420
1,2024-09-21,TCS.NS,PE,4300,2024-09-26,0.234269,0.228705,3.0,4295.000000,43.737666,42.697629,-0.494480,0.003634
2,2024-09-21,WIPRO.NS,CE,540,2024-09-26,0.357009,0.319815,3.0,538.200012,7.806048,6.935395,0.485770,0.019017
3,2024-09-21,SBIN.NS,PE,780,2024-09-26,0.218513,0.215797,3.0,784.500000,5.034838,4.946295,-0.380892,0.020372
